# Wikipedia 한국어 데이터셋 다운로드 및 벡터 인덱싱

이 노트북에서는 Hugging Face Datasets에서 한국어 Wikipedia 데이터셋을 다운로드하고, Pinecone 벡터 데이터베이스에 인덱싱하는 과정을 진행합니다.


## 1. 필요한 라이브러리 설치 및 임포트


In [ ]:
# 필요한 라이브러리 설치 및 임포트
%pip install datasets

from datasets import load_dataset
import pandas as pd
import json
from tqdm import tqdm
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


## 2. Wikipedia 한국어 데이터셋 다운로드


In [ ]:
# Wikipedia 한국어 데이터셋 다운로드
print("=== Wikipedia 한국어 데이터셋 다운로드 시작 ===")

try:
    # 데이터셋 로드 (처음에는 시간이 오래 걸릴 수 있습니다)
    print("데이터셋 로딩 중... (처음 실행 시 다운로드가 필요합니다)")
    dataset = load_dataset("wikimedia/wikipedia", "20231101.ko")
    
    print(f"✅ 데이터셋 로드 완료!")
    print(f"📊 데이터셋 정보:")
    print(f"   - 훈련 데이터: {len(dataset['train']):,}개 문서")
    print(f"   - 컬럼: {dataset['train'].column_names}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    dataset = None


In [ ]:
# 샘플 데이터 확인
if dataset:
    print("📝 샘플 문서들:")
    
    for i in range(3):
        sample = dataset['train'][i]
        print(f"\n--- 문서 {i+1} ---")
        print(f"ID: {sample.get('id', 'N/A')}")
        print(f"제목: {sample.get('title', 'N/A')}")
        print(f"텍스트 길이: {len(sample.get('text', '')):,}자")
        print(f"텍스트 미리보기: {sample.get('text', '')[:300]}...")


## 3. 샘플 데이터 저장 및 분석


In [ ]:
# 샘플 데이터를 로컬에 저장
if dataset:
    print("=== 샘플 데이터 저장 중 ===")
    
    # 1000개 샘플 추출
    num_samples = 1000
    sample_data = dataset['train'].select(range(min(num_samples, len(dataset['train']))))
    
    # CSV로 저장
    sample_data.to_csv('wikipedia_ko_sample.csv')
    print(f"✅ CSV 파일 저장 완료: wikipedia_ko_sample.csv ({len(sample_data)}개 문서)")
    
    # JSON으로 저장
    sample_data.to_json('wikipedia_ko_sample.json')
    print(f"✅ JSON 파일 저장 완료: wikipedia_ko_sample.json ({len(sample_data)}개 문서)")
    
    # 통계 정보
    print(f"\n📊 샘플 데이터 통계:")
    print(f"   - 총 문서 수: {len(sample_data):,}개")
    print(f"   - 평균 텍스트 길이: {sum(len(doc['text']) for doc in sample_data) // len(sample_data):,}자")
    print(f"   - 최대 텍스트 길이: {max(len(doc['text']) for doc in sample_data):,}자")
    print(f"   - 최소 텍스트 길이: {min(len(doc['text']) for doc in sample_data):,}자")


In [ ]:
# 특정 주제로 필터링 예제
if dataset:
    print("=== 특정 주제로 필터링 ===")
    
    # '인공지능' 관련 문서 찾기
    ai_docs = []
    for i, doc in enumerate(tqdm(dataset['train'], desc="검색 중...")):
        if '인공지능' in doc.get('title', '') or '인공지능' in doc.get('text', ''):
            ai_docs.append(doc)
            if len(ai_docs) >= 5:  # 5개만 찾기
                break
    
    print(f"\n🤖 '인공지능' 관련 문서 {len(ai_docs)}개 발견:")
    for i, doc in enumerate(ai_docs):
        print(f"{i+1}. {doc.get('title', 'N/A')}")
        print(f"   텍스트 길이: {len(doc.get('text', '')):,}자")
        print(f"   미리보기: {doc.get('text', '')[:100]}...")
        print()


## 4. OpenAI API 설정 및 임베딩 생성


In [ ]:
# OpenAI API 설정
openai.api_key = OPENAI_API_KEY

def get_embedding(text, model="text-embedding-3-small"):
    """텍스트를 임베딩으로 변환"""
    try:
        response = openai.embeddings.create(
            input=text,
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        return None

# 테스트
test_embedding = get_embedding("안녕하세요 테스트입니다")
if test_embedding:
    print(f"✅ 임베딩 테스트 성공! 차원: {len(test_embedding)}")
else:
    print("❌ 임베딩 테스트 실패")


## 5. Pinecone 벡터 데이터베이스 설정


## 6. Wikipedia 데이터를 벡터 데이터베이스에 인덱싱


In [ ]:
# Wikipedia 데이터를 벡터로 변환하고 Pinecone에 저장
if dataset and test_embedding:
    print("=== Wikipedia 데이터 벡터화 및 인덱싱 시작 ===")
    
    # 샘플 데이터로 시작 (처음에는 작은 규모로 테스트)
    sample_size = 100
    sample_docs = dataset['train'].select(range(min(sample_size, len(dataset['train']))))
    
    vectors_to_upsert = []
    
    for i, doc in enumerate(tqdm(sample_docs, desc="벡터화 중...")):
        # 텍스트 전처리 (너무 긴 텍스트는 잘라내기)
        text = doc.get('text', '')
        if len(text) > 8000:  # OpenAI 임베딩 모델의 토큰 제한 고려
            text = text[:8000]
        
        # 임베딩 생성
        embedding = get_embedding(text)
        
        if embedding:
            vectors_to_upsert.append({
                'id': f"wiki_{doc.get('id', i)}",
                'values': embedding,
                'metadata': {
                    'title': doc.get('title', ''),
                    'text_length': len(doc.get('text', '')),
                    'source': 'wikipedia_ko'
                }
            })
    
    print(f"✅ {len(vectors_to_upsert)}개 벡터 생성 완료")
    
    # Pinecone에 벡터 저장
    if vectors_to_upsert:
        print("Pinecone에 벡터 저장 중...")
        index.upsert(vectors=vectors_to_upsert)
        print("✅ 벡터 저장 완료!")
        
        # 인덱스 통계 확인
        stats = index.describe_index_stats()
        print(f"📊 인덱스 통계: {stats}")
    else:
        print("❌ 저장할 벡터가 없습니다.")


## 7. 벡터 검색 테스트


In [ ]:
# 벡터 검색 테스트
def search_wikipedia(query, top_k=5):
    """Wikipedia 데이터에서 유사한 문서 검색"""
    # 쿼리를 임베딩으로 변환
    query_embedding = get_embedding(query)
    
    if not query_embedding:
        print("❌ 쿼리 임베딩 생성 실패")
        return None
    
    # Pinecone에서 검색
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    return results

# 검색 테스트
test_queries = [
    "인공지능의 역사",
    "한국의 전통 음식",
    "과학 기술 발전",
    "경제 성장과 발전"
]

print("=== 벡터 검색 테스트 ===")
for query in test_queries:
    print(f"\n🔍 검색 쿼리: '{query}'")
    results = search_wikipedia(query, top_k=3)
    
    if results and results.matches:
        for i, match in enumerate(results.matches):
            print(f"  {i+1}. {match.metadata.get('title', 'N/A')} (유사도: {match.score:.3f})")
            print(f"     텍스트 길이: {match.metadata.get('text_length', 0):,}자")
    else:
        print("  검색 결과가 없습니다.")


## 8. RAG 시스템 구축을 위한 데이터셋 사용법

💡 이제 Wikipedia 한국어 데이터셋을 사용할 수 있습니다:

### 1. 전체 데이터셋 사용
```python
dataset = load_dataset('wikimedia/wikipedia', '20231101.ko')
docs = dataset['train']
```

### 2. 특정 문서 접근
```python
doc = dataset['train'][0]  # 첫 번째 문서
title = doc['title']
text = doc['text']
```

### 3. 샘플 데이터 사용
```python
import pandas as pd
df = pd.read_csv('wikipedia_ko_sample.csv')
```

### 4. RAG 시스템에 활용
- 텍스트를 청크로 분할
- 임베딩 생성
- 벡터 데이터베이스에 저장
- 검색 및 생성에 활용


In [ ]:
# 'wiki' index 생성 - 1536 dim

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [4]:
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-506yx3r.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# data load

In [6]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [9]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

In [10]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [11]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [12]:
len(data)

100

In [13]:
for rec in data:
  if len(rec['text'])>35000:
    print('**')

**
**


In [14]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001CD5CEBBA40>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001CD5EA37590>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=400,
  chunk_overlap =20,
  length_function = len, \
  separators=["\n\n", "\n", " ", ""]  #자르는 순서 문자
)
splitter

In [19]:
import json
def calculate_metadata_size(metadata):
  return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [22]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [ ]:
for i, sample in enumerate(data):
  full_text = sample['text'] #전문
  
  #메타데이터 구성
  metadata = {
    "wiki_id": str(sample['id']),
    "url": sample['url'],
    "title": sample['title'] #[:100]
  }
  
  # 스플릿터로 자르기
  chunks = splitter.split_text(full_text)
  #print(len(chunks))
  
  for i, chunk in enumerate(chunks):
    record = {
      'chunk_id':i,
      'full_text':full_text,
      **metadata
    }
    metadata_size = calculate_metadata_size(record)
    if metadata_size > 35000:
      continue
    
    texts.append(chunk)
    metas.append(record)
    count += 1
    
    # 청크 배치의 임베딩 -> 적재
    if count % batch_size == 0:
      ids = [str(uuid4()) for _ in range(len(texts))]
      embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
      index.upsert(
        vectors=zip(ids, embeddings, metas),
        namespace='wiki-ns1'
      )
      texts=[]
      metas=[]
      time.sleep(1)